<a href="https://colab.research.google.com/github/aryabhat1/mlwork/blob/master/4_cat_vs_dog_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import shutil

work_dir = '/content'
image_names = sorted(os.listdir(os.path.join(work_dir, 'mydata')))


def copy_files(prefix_str, range_start, range_end, target_dir):
    image_paths = [os.path.join(work_dir, 'mydata', prefix_str + '.' + str(i) + '.jpg')
                   for i in range(range_start, range_end)]
    dest_dir = os.path.join(work_dir, 'data', target_dir, prefix_str)
    os.makedirs(dest_dir)
    for image_path in image_paths:
        shutil.copy(image_path, dest_dir)


copy_files('dog', 0, 210, 'train')
copy_files('cat', 0, 210, 'train')
copy_files('dog', 211, 300, 'test')
copy_files('cat', 211, 300, 'test')


In [11]:
import tensorflow as tf
import numpy as np

In [0]:
image_height, image_width = 150, 150
train_dir = os.path.join(work_dir, 'data/train')
test_dir = os.path.join(work_dir, 'data/test')
no_classes = 2
no_validation = 80
epochs = 2
batch_size = 20
no_train = 200
no_test = 80
input_shape = (image_height, image_width, 3)
epoch_steps = no_train // batch_size
test_steps = no_test // batch_size

In [13]:
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

simple_cnn_model = simple_cnn(input_shape)

generator_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
generator_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

train_images = generator_train.flow_from_directory(
    train_dir,
    batch_size=batch_size,
    target_size=(image_width, image_height))

test_images = generator_test.flow_from_directory(
    test_dir,
    batch_size=batch_size,
    target_size=(image_width, image_height))

simple_cnn_model.fit_generator(
    train_images,
    steps_per_epoch=epoch_steps,
    epochs=epochs,
    validation_data=test_images,
    validation_steps=test_steps)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Found 420 images belonging to 2 classes.
Found 178 images belonging to 2 classes.
Epoch 1/2
10/10 [==============================] - 77s 8s/step - loss: 27.0758 - acc: 0.5550 - val_loss: 0.7178 - val_acc: 0.4875
Epoch 2/2
10/10 [==============================] - 57s 6s/step - loss: 0.6983 - acc: 0.5050 - val_loss: 0.6920 - val_acc: 0.5250
